# ProcessCcd 

<br>Owner: **Alex Drlica-Wagner** ([@kadrlica](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@kadrlica))
<br>Last Verified to Run: **2018-08-10**
<br>Verified Stack Release: **v16.0**

## Learning Objectives:

This notebook seeks to teach users how to unpack a command line task, specifically `processCcd.py`, to get at the python API functionality that is being called. It is a digression from Justin Myles implementation of the HSC rerun processing [link].

After working through this tutorial you should be able to:

* Find the source code for a command line task
* Investigate and run those tasks in python

## Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

In [ ]:
import os

## Command Line Processing

In this first step we perform command line processing of images following the getting started tutorials [here](https://pipelines.lsst.io/getting-started/data-setup.html#) and [here](https://pipelines.lsst.io/getting-started/processccd.html). We are specifically interested in digging into the following line:

```
processCcd.py $DATADIR --rerun processCcdOutputs --id
```

We start by tracking down the location of the `processCcd.py` shell script

In [1]:
!(which processCcd.py)

/opt/lsst/software/stack/stack/miniconda3-4.3.21-10a4fa6/Linux64/pipe_tasks/16.0+1/bin/processCcd.py


This is the proverbial "end of the thread". Our goal is to pull on this thread to unravel the python/C++ functions that are being called under the hood. We start by taking a peak in this script

In [2]:
!cat $(which processCcd.py)

#!/usr/bin/env python
#
# LSST Data Management System
# Copyright 2008, 2009, 2010 LSST Corporation.
#
# This product includes software developed by the
# LSST Project (http://www.lsst.org/).
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.    See the
# GNU General Public License for more details.
#
# You should have received a copy of the LSST License Statement and
# the GNU General Public License along with this program.  If not,
# see <http://www.lsstcorp.org/LegalNotices/>.
#
from lsst.pipe.tasks.processCcd import ProcessCcdTask

ProcessCcdTask.parseAndRun()


Ok, this hasn't gotten us very far, but we now have the next link in our chain:
```
from lsst.pipe.tasks.processCcd import ProcessCcdTask
```
This seems promising! 

There are two ways we can proceed from here. One is to Google `lsst.pipe.tasks.processCcd`, which will take us to this [doxygen page](http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/classlsst_1_1pipe_1_1tasks_1_1process_ccd_1_1_process_ccd_task.html) and/or this soure code on [GitHub](https://github.com/lsst/pipe_tasks/blob/master/python/lsst/pipe/tasks/processCcd.py). The second approach is to do the import oursleves...


In [7]:
import lsst.pipe.tasks.processCcd
from lsst.pipe.tasks.processCcd import ProcessCcdTask, ProcessCcdConfig

In [5]:
#help(lsst.pipe.tasks.processCcd)

In [13]:
c = ProcessCcdConfig()
c.formatHistory('*')

KeyError: '*'

We'd like to investigate what data we have available (the `--show-data` flag).

In [ ]:
!processCcd.py DATA --rerun processCcdOutputs --id --show data

We choose one CCD `(visit=903334, ccd=16)` and pass it to the `processCcd` command line task

In [ ]:
!processCcd.py DATA --rerun processCcdOutputs --id visit=903334 ccd=16